In [1]:
import os

In [7]:
os.chdir('../')

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [10]:
from MLOps_Test.constants import *
from MLOps_Test.utils.common import read_yaml,create_directories

Configuration class to read yaml files

In [22]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

Update data ingestion components

In [12]:
import os
import urllib.request as request
import zipfile
from MLOps_Test import logger
from MLOps_Test.utils.common import get_size

In [13]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(url = self.config.source_URL, filename=self.config.local_data_file)
            logger.info(f"{filename} downloaded file with following info \n{headers} ")
        else:
            logger.info(f"File aldready exists of size : {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

Pipeline for Data Ingestion

In [23]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-07 18:00:21,223: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-07 18:00:21,224: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-07 18:00:21,225: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-07 18:00:21,225: INFO: common: created directory at: artifacts]
[2024-02-07 18:00:21,226: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-07 18:00:21,227: INFO: 224548410: File aldready exists of size : ~ 23 KB]
